# ScikitLearnの学習
機械学習・データマイニングを行うためのライブラリ。
かなり複雑な深層学習以外のMLはたいていScikitで実行できる
numpy・pandasと合わせて使うのが前提

## 前処理
Scikitではデータの前処理を行う事ができる
- 欠損値の対処
- 定性データをワンホットベクトルに変換
- 定量データの正規化・正則化

###　欠損値の除去・補完

In [9]:
import numpy as np
import pandas as pd

df = pd.DataFrame(
    {
    "A":[1,np.nan,3,4,5],
    "B":[6,7,8,np.nan,10],
    "C":[11,12,13,14,15]
    }
)

df

,A,B,C
0,1.0,6.0,11
1,NaN,7.0,12
2,3.0,8.0,13
3,4.0,NaN,14
4,5.0,10.0,15


In [10]:
# 欠損値かをbool値で返す
df.isnull()

,A,B,C
0,False,False,False
1,True,False,False
2,False,False,False
3,False,True,False
4,False,False,False


In [12]:
# 欠損値の保管 - 値を補完する、SimpleImputerのメソッドを用いる
# mean-平均、median-中央値、most_frequent-最頻値、constant-定数
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy="mean")
imp.fit(df)
imp.transform(df)

array([[ 1.  ,  6.  , 11.  ],
       [ 3.25,  7.  , 12.  ],
       [ 3.  ,  8.  , 13.  ],
       [ 4.  ,  7.75, 14.  ],
       [ 5.  , 10.  , 15.  ]])

In [13]:
from sklearn.impute import SimpleImputer
value = 0

imp = SimpleImputer(strategy="constant", fill_value=value)
imp.fit(df)
imp.transform(df)

array([[ 1.,  6., 11.],
       [ 0.,  7., 12.],
       [ 3.,  8., 13.],
       [ 4.,  0., 14.],
       [ 5., 10., 15.]])

### 定性データをワンホットベクトルに変換

In [15]:
df = pd.DataFrame(
    {
    "A":[1,2,3,4,5],
    "B":["a","b","c","a","b"]
    }
)

df

,A,B
0,1,a
1,2,b
2,3,c
3,4,a
4,5,b


In [20]:
# LabelEncoder - str型などの定性データを値ごとにワンホットベクトルに変換する
from sklearn.preprocessing import LabelEncoder

#エンコーディング
le = LabelEncoder()
le.fit(df.loc[:,"B"])

#変換後・元の値を確認
print(le.transform(df.loc[:,"B"]))
print(le.classes_ )

[0 1 2 0 1]
['a' 'b' 'c']


In [22]:
# LabelEncoder - str型などの定性データを値ごとにワンホットベクトルに変換する
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.compose import ColumnTransformer

#データフレームを複製
df_ohe = df.copy()

#エンコーディング
le = LabelEncoder()
df_ohe.loc[:,"B"] = le.fit_transform(df_ohe.loc[:,"B"])

ohe = ColumnTransformer([("OneHotEncoder", OneHotEncoder(),[1])], remainder="passthrough")

df_ohe = ohe.fit_transform(df_ohe)
df_ohe

C:\Users\guest017\AppData\Local\Temp\ipykernel_13100\922263664.py:10: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_ohe.loc[:,"B"] = le.fit_transform(df_ohe.loc[:,"B"])


array([[1., 0., 0., 1.],
       [0., 1., 0., 2.],
       [0., 0., 1., 3.],
       [1., 0., 0., 4.],
       [0., 1., 0., 5.]])

### 定量データの正規化・正則化
- 分散正規化-平均が0，標準偏差を1にする
- 最小最大正規化 - 最小値が0、最大値を1にする

In [29]:
#分散正規化
import pandas as pd

df = pd.DataFrame(
    {
    "A":[1,2,3,4,5],
    "B":[100,200,400,500,800]
    }
)

df

,A,B
0,1,100
1,2,200
2,3,400
3,4,500
4,5,800


In [30]:
#StandardScaler - 分散正規化のインスタンス
from sklearn.preprocessing import StandardScaler

#インスタンスを作成→fit、transformで正規化を実行する
stdsc = StandardScaler()

stdsc.fit(df)
stdsc.transform(df)

array([[-1.41421356, -1.22474487],
       [-0.70710678, -0.81649658],
       [ 0.        ,  0.        ],
       [ 0.70710678,  0.40824829],
       [ 1.41421356,  1.63299316]])

In [31]:
#MinMaxScaler - 最大最小正規化のインスタンス
from sklearn.preprocessing import MinMaxScaler

#インスタンスを作成→fit、transformで正規化を実行する
mmsc = MinMaxScaler()

mmsc.fit(df)
mmsc.transform(df)

array([[0.        , 0.        ],
       [0.25      , 0.14285714],
       [0.5       , 0.42857143],
       [0.75      , 0.57142857],
       [1.        , 1.        ]])

## 分類アルゴリズム
データをクラスに分類する。大量のデータを分類するには、機械学習の中でも教師あり学習が活用される。
ユーザーを利用履歴などをもとに購入層ごと、またはサービスから退会する可能性を予測するのに用いる。

- サポートベクターマシン
- 決定木
- ランダムフォレスト法

分類用のモデルを作成するには、まずデータを学習用・テスト用に分割する。
機械学習の目的は学習用のデータだけに適応するだけでなく、テスト用のデータのような未知の値に対応する性能（汎化能力）を向上することである。
汎化能力を効率よく向上させる方法として、学習・テストデータの分割とモデル構築を複数回繰り返す交差検証が活用されている。

今回は有名なirisデータセットを使って分類を行う